In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [4]:
weekly_aggregated_df = pd.DataFrame()
temp_df = pd.DataFrame()
current_monday = None
current_sunday = None
t = 0

for year in range(2013, 2024, 1):
    print(f'I started year {year}')
    
    df = pd.read_csv(f'ProcessedCitibikeData/ProcessedCitibikeData/CitibikeProcessed{year}.csv')

    df = df[~((df['start_zone_encoded'] == 264) | (df['start_zone_encoded'] == 265) | (df['end_zone_encoded'] == 264) | (df['end_zone_encoded'] == 265))]
    df['start_time'] = pd.to_datetime(df['unix_start_time'], unit='s')
    df['stop_time'] = pd.to_datetime(df['unix_stop_time'], unit='s')
    df = df[['start_zone_encoded', 'end_zone_encoded', 'start_time', 'stop_time']]
    df['weight'] = 0
    
    df = pd.concat([temp_df, df], ignore_index=True)
    df = df.sort_values(by = 'start_time').reset_index(drop=True)
    temp_df = pd.DataFrame()
    
    if year == 2013:
        first_monday = df.loc[df['start_time'].dt.dayofweek == 0].iloc[0]['start_time']
        current_monday = first_monday.normalize()
        current_sunday = (current_monday + pd.Timedelta(days=7)).normalize()

    while current_monday <= df['stop_time'].max():
        # Slice DataFrame for the current week (Monday to Sunday)
        current_sunday = (current_monday + pd.Timedelta(days=7)).normalize()
        if current_sunday > df['stop_time'].max():
            temp_df = df[(df['start_time'] >= current_monday)]
            break
        
        weekly_df = df[(df['start_time'] >= current_monday) & (df['start_time'] <= current_sunday)]
        
        total_df = weekly_df.groupby(['start_zone_encoded', 'end_zone_encoded']).agg({'start_time':'min', 'stop_time':'max', 'weight':'count'}).reset_index()
        total_df['time'] = t
        weekly_aggregated_df = pd.concat([weekly_aggregated_df, total_df], ignore_index=True)
        
        # Move to next Monday
        current_monday = (current_monday + pd.Timedelta(days=7)).normalize()
        t += 1

I started year 2013
I started year 2014
I started year 2015
I started year 2016
I started year 2017
I started year 2018
I started year 2019
I started year 2020
I started year 2021
I started year 2022
I started year 2023


In [5]:
weekly_aggregated_df

,start_zone_encoded,end_zone_encoded,start_time,stop_time,weight,time
0,4,12,2013-06-04 18:45:34,2013-06-05 10:38:47,2,0
1,4,13,2013-06-03 18:03:16,2013-06-09 21:07:51,15,0
2,4,17,2013-06-08 16:10:30,2013-06-08 16:26:21,1,0
3,4,25,2013-06-05 16:45:02,2013-06-08 18:35:33,3,0
4,4,33,2013-06-03 17:47:24,2013-06-09 18:34:05,13,0
...,...,...,...,...,...,...
3134446,263,247,2023-12-28 15:10:15,2023-12-28 15:39:14,1,551
3134447,263,249,2023-12-25 13:45:23,2023-12-29 15:17:34,6,551
3134448,263,256,2023-12-25 01:27:22,2023-12-31 13:22:27,3,551
3134449,263,260,2023-12-27 14:35:08,2023-12-28 15:10:16,2,551


In [6]:
weekly_aggregated_df = weekly_aggregated_df.drop(['start_time', 'stop_time'], axis=1)

weekly_aggregated_df

,start_zone_encoded,end_zone_encoded,weight,time
0,4,12,2,0
1,4,13,15,0
2,4,17,1,0
3,4,25,3,0
4,4,33,13,0
...,...,...,...,...
3134446,263,247,1,551
3134447,263,249,6,551
3134448,263,256,3,551
3134449,263,260,2,551


In [7]:
weekly_aggregated_df.to_csv('weeklyAggregatedCitibike_for_train.csv', index=False)